# Lateral ventricle volume trajectories and response inhibition - lda

2021, Arvid Lundervold

*Astri J. Lundervold, Alexandra Vik, Arvid Lundervold* <br>
**Lateral ventricle volume trajectories predict response inhibition in older age - a
longitudinal brain imaging and machine learning approach**<br>  (to appear in PLOS ONE) <br>

The linear mixed effect model (LME) was fitted to the data, i.e.

$$\text{Vol}_{ij} = \beta_0 + \beta_1 \text{Age}_{ij} + (b_{0i} + b_{1i} \text{Age}_{ij}) + \epsilon_{ij}$$

Here, $\text{Vol}_{ij}$ the continuous *response variable* in the model is volume of left (right) latreral ventricle in subject $i$ ($i=1,\ldots,N$) at wave $j$ ($j=1,\ldots,n_i$). In our case we have $N=74$ and three wase with complete data, i.e. $n_i=3$ for all $i$. $\text{Age}_{ij}$ is age (in years) of subject $j$ at wave $j$, and a *predictor variable* in the model.

The model parameterrs $\beta_0$ and $\beta_1$ are *fixed effects* parameters.  The variables $b_{0i}$ and $b_{1i}$ 
($i=1,\ldots,N$) are the *random effects* parameters, assumed to be normally distributed witrh zero mean. They denote individual deviations in intercept ($b_{0i}$) and slope ($b_{1i}$), respectiveley, from the group-level fixed effect.
Finally, the random residual errors $\epsilon_{ij}$ are assumed to be independent and normally distributed (i.i.d) with zero mean and constant variance $\sigma_\epsilon^2$.

**Select if eTIV-normalization of lateral ventricle volumes should be used or not**

In [1]:
eTIV_NORMALIZED = False

### Packages and libraries
**`rpy2` should be tested through `0.0-test.ipynb`**

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import math
from matplotlib import pyplot as plt
#import rpy2
#from rpy2.robjects import r, pandas2ri
#pandas2ri.activate()
#from rpy2.robjects.lib.tidyr import DataFrame

# Enable inline plotting
%matplotlib inline
from IPython.display import Image

## Reading data

In [3]:
fn_data = '../data/01_lvv_ri_renamed_data.csv'
df = pd.read_csv(fn_data)

In [4]:
df.head()

,subj,gender,yrW1,yrW2,yrW3,left_lvvW1,left_lvvW2,left_lvvW3,right_lvvW1,right_lvvW2,right_lvvW3,eTIV,RI
0,subj_01,F,56.63,60.35,62.62,5321.4,5063.6,5368.3,6855.2,6729.2,7233.2,1.232679e+06,66.0
1,subj_02,M,49.07,52.58,55.05,11038.6,11912.2,12313.3,10611.6,11774.8,12070.5,1.464692e+06,85.0
2,subj_03,M,74.61,78.14,80.59,29718.4,32265.1,35887.0,34241.0,37155.2,41221.4,1.385841e+06,71.0
3,subj_04,M,56.23,59.78,62.68,21830.1,23933.6,26164.2,15243.9,16505.1,18509.1,1.529445e+06,41.0
4,subj_05,M,63.22,67.00,69.53,17205.1,16970.1,15858.1,13962.1,13755.5,13143.9,1.371460e+06,62.0


In [5]:
df.to_csv('../data/02_lvv_ri_new_data_wide.csv', encoding='utf-8', index=False)

In [6]:
#if eTIV_NORMALIZED:
df_nor = df.copy()
for col in ['left_lvvW1', 'left_lvvW2', 'left_lvvW3',
                'right_lvvW1', 'right_lvvW2','right_lvvW3']:
        df_nor[col] = df[col]/df['eTIV']

df_nor.head()

,subj,gender,yrW1,yrW2,yrW3,left_lvvW1,left_lvvW2,left_lvvW3,right_lvvW1,right_lvvW2,right_lvvW3,eTIV,RI
0,subj_01,F,56.63,60.35,62.62,0.004317,0.004108,0.004355,0.005561,0.005459,0.005868,1.232679e+06,66.0
1,subj_02,M,49.07,52.58,55.05,0.007536,0.008133,0.008407,0.007245,0.008039,0.008241,1.464692e+06,85.0
2,subj_03,M,74.61,78.14,80.59,0.021444,0.023282,0.025895,0.024708,0.026811,0.029745,1.385841e+06,71.0
3,subj_04,M,56.23,59.78,62.68,0.014273,0.015649,0.017107,0.009967,0.010792,0.012102,1.529445e+06,41.0
4,subj_05,M,63.22,67.00,69.53,0.012545,0.012374,0.011563,0.010180,0.010030,0.009584,1.371460e+06,62.0


In [7]:
df_nor.to_csv('../data/02_lvv_ri_new_eTIV_norm_data_wide.csv', encoding='utf-8', index=False)

 ## Prepare for using lmer in R for linear mixed-effects (LME) analysis
 
  See `02_lvv_ri_lda_R.ipynb` (with the R kernel)
  
#### A note on R (tip for previous R users)
It is possible to run R-scripts in Jupyter notebooks (Jupyter = Julia, Python and R).

If you want to continue working with R (not part of this course) you should:

- Be using the latest R version (https://www.r-project.org) and the RStudio Desktop [download] for your Windows, MacOS or Linux system.
- Install the R kernel by opening an R console and then follow the instructions at https://irkernel.github.io/installation
- Necessary (or new) R libraries should be installed via RStudio and the corresponding R environment.
- See also [here](https://datatofish.com/r-jupyter-notebook) and [here](https://developers.refinitiv.com/en/article-catalog/article/setup-jupyter-notebook-r).

The [rpy2](https://github.com/rpy2/rpy2) interface to use R form Python is also possible (but can be a bit more messy).

 **`rpy2` should be tested through `0.0-test.ipynb`**

In [8]:
if eTIV_NORMALIZED:
    df_lmer = pd.read_csv('../results/02_lvv_ri_new_R_long_data_and_features_eTIV_norm.csv')
else:
    df_lmer = pd.read_csv('../results/02_lvv_ri_new_R_long_data_and_features.csv')

In [9]:
df_lmer.head().T.round(2)

,0,1,2,3,4
id,1,2,3,4,5
Gender,F,M,M,M,M
Age1,56.63,49.07,74.61,56.23,63.22
Age2,60.35,52.58,78.14,59.78,67.0
Age3,62.62,55.05,80.59,62.68,69.53
b0iL,15783.971789,11457.990229,-27873.543885,-3973.21534,35796.880807
b1iL,-403.106006,-192.279528,504.164116,242.654739,-535.067249
beta0plusb0iL,3726.35047,-599.63109,-39931.165204,-16030.836658,23739.259488
beta1plusb1iL,25.621259,236.447737,932.891381,671.382003,-106.339985
b0iR,9303.751654,579.808396,-16362.882836,-4710.959539,5763.12965


In [10]:
df_lmer.describe().T.round(4)

,count,mean,std,min,25%,50%,75%,max
id,74.0,37.5000,21.5058,1.0000,19.2500,37.5000,55.7500,74.0000
Age1,74.0,60.7482,7.2548,46.6600,55.6125,59.8750,66.3925,77.6300
Age2,74.0,64.2839,7.2110,50.1100,59.3175,63.5550,69.2300,81.3300
Age3,74.0,66.8126,7.1855,52.4600,61.8600,66.0200,72.0925,84.0500
b0iL,74.0,-0.0000,19104.1076,-92438.6924,-6894.5245,3257.9935,10096.4873,35796.8808
b1iL,74.0,0.0000,370.0545,-736.2918,-219.9263,-84.9836,213.9539,1605.3663
beta0plusb0iL,74.0,-12057.6213,19104.1076,-104496.3137,-18952.1458,-8799.6278,-1961.1340,23739.2595
beta1plusb1iL,74.0,428.7273,370.0545,-307.5645,208.8009,343.7437,642.6811,2034.0936
b0iR,74.0,0.0000,8988.7901,-32640.0136,-3525.8863,2838.3733,6388.6144,12136.8345
b1iR,74.0,-0.0000,248.1606,-340.3251,-170.6087,-72.1386,96.2811,846.8899


In [11]:
dff = df_lmer.copy()
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             74 non-null     int64  
 1   Gender         74 non-null     object 
 2   Age1           74 non-null     float64
 3   Age2           74 non-null     float64
 4   Age3           74 non-null     float64
 5   b0iL           74 non-null     float64
 6   b1iL           74 non-null     float64
 7   beta0plusb0iL  74 non-null     float64
 8   beta1plusb1iL  74 non-null     float64
 9   b0iR           74 non-null     float64
 10  b1iR           74 non-null     float64
 11  beta0plusb0iR  74 non-null     float64
 12  beta1plusb1iR  74 non-null     float64
 13  LatVentL1      74 non-null     float64
 14  LatVentL2      74 non-null     float64
 15  LatVentL3      74 non-null     float64
 16  LatVentR1      74 non-null     float64
 17  LatVentR2      74 non-null     float64
 18  LatVentR3   

## Model-based (lmer) feature design

![Fig_2_LME_derived_features](./figs/Fig_2_LME_derived_features.png)

## Derive new feature, related to wave1 (baseline)

In [12]:
# Derive new feature, related to wave1

def volFixed(x, a_f, b_f):
    '''
    Age at wave 1: x = dff.Age1
      a_f = dff.beta1L = dff.beta1plusb1iL - dff.b1iL
      b_f = dff.beta0plusb0iL - dff.b0iL
    Fixed effects: y_f = a_f x + b_f
    '''
    return a_f*x + b_f

    
def volDeviation(x, a_f, b_f, a_r, b_r):
    '''
    Age at wave 1: x = dff.Age1
    Fixed effects: y_f = a_f x + b_f
      a_f = dff.beta1L = dff.beta1plusb1iL - dff.b1iL
      b_f = dff.beta0plusb0iL - dff.b0iL
    Random effects: y_r = a_r x + b_r
      a_r = dff.beta1plusb1iL
      b_r = dff.beta0plusb0iL
    Vol_deviation = y_r - y_f
    '''

    y_f = a_f*x + b_f
    y_r = a_r*x + b_r
    return y_r - y_f

**Left:**

In [13]:
x = dff.Age1

# Left

a_f = dff['beta1plusb1iL'].values - dff['b1iL'].values
b_f = dff['beta0plusb0iL'].values - dff['b0iL'].values
a_r = dff['beta1plusb1iL'].values
b_r = dff['beta0plusb0iL'].values

print('LH fixed effects: (a_f, b_f) = (beta1L, beta0L) = (%.4f, %.4f)' % (a_f[0], b_f[0]))
dff['VdevL'] = volDeviation(x, a_f, b_f, a_r, b_r)
dff['Vfixed1L'] = volFixed(x, a_f, b_f)
dff['Vfixed1diffL'] = dff['LatVentL1'] - dff['Vfixed1L']

LH fixed effects: (a_f, b_f) = (beta1L, beta0L) = (428.7273, -12057.6213)


**Right:**

In [14]:
# Right

a_f = dff['beta1plusb1iR'].values - dff['b1iR'].values
b_f = dff['beta0plusb0iR'].values - dff['b0iR'].values
a_r = dff['beta1plusb1iR'].values
b_r = dff['beta0plusb0iR'].values

print('RH fixed effects: (a_f, b_f) = (beta1R, beta0R) = (%.4f, %.4f)' % (a_f[0], b_f[0]))
dff['VdevR'] = volDeviation(x, a_f, b_f, a_r, b_r)
dff['Vfixed1R'] = volFixed(x, a_f, b_f)
dff['Vfixed1diffR'] = dff['LatVentR1'] - dff['Vfixed1R']

RH fixed effects: (a_f, b_f) = (beta1R, beta0R) = (408.6075, -11482.2242)


In [15]:
pd.DataFrame(dff[['id', 'Age1', 'b0iL', 'b1iL', 'LatVentL1', 'Vfixed1L', 'Vfixed1diffL', 'VdevL']].round(4)).head()

,id,Age1,b0iL,b1iL,LatVentL1,Vfixed1L,Vfixed1diffL,VdevL
0,1,56.63,15783.9718,-403.1060,5321.4,12221.2037,-6899.8037,-7043.9213
1,2,49.07,11457.9902,-192.2795,11038.6,8980.0256,2058.5744,2022.8338
2,3,74.61,-27873.5439,504.1641,29718.4,19929.7199,9788.6801,9742.1408
3,4,56.23,-3973.2153,242.6547,21830.1,12049.7128,9780.3872,9671.2606
4,5,63.22,35796.8808,-535.0672,17205.1,15046.5163,2158.5837,1969.9293


In [16]:
dff['errorL'] = dff['Vfixed1diffL'] - dff['VdevL']

In [17]:
pd.DataFrame(dff[['id', 'Age1', 'b0iR', 'b1iR', 'LatVentR1', 'Vfixed1R', 'Vfixed1diffR', 'VdevR']].round(4)).head()

,id,Age1,b0iR,b1iR,LatVentR1,Vfixed1R,Vfixed1diffR,VdevR
0,1,56.63,9303.7517,-255.8375,6855.2,11657.2207,-4802.0207,-5184.3248
1,2,49.07,579.8084,18.4452,10611.6,8568.1477,2043.4523,1484.9150
2,3,74.61,-16362.8828,431.9485,34241.0,19003.9843,15237.0157,15864.7936
3,4,56.23,-4710.9595,143.7001,15243.9,11493.7777,3750.1223,3369.2981
4,5,63.22,5763.1296,-119.3152,13962.1,14349.9444,-387.8444,-1779.9775


In [18]:
dff['errorR'] = dff['Vfixed1diffR'] - dff['VdevR']

In [19]:
pd.DataFrame(dff[['Age1', 'b0iL', 'b1iL', 'LatVentL1', 'Vfixed1L', 'Vfixed1diffL', 'VdevL', 'errorL']].mean().round(5))

,0
Age1,60.74824
b0iL,-0.00000
b1iL,0.00000
LatVentL1,14994.09730
Vfixed1L,13986.80683
Vfixed1diffL,1007.29047
VdevL,1002.70636
errorL,4.58410


In [20]:
pd.DataFrame(dff[['Age1', 'b0iR', 'b1iR', 'LatVentR1', 'Vfixed1R', 'Vfixed1diffR', 'VdevR', 'errorR']].mean().round(5))

,0
Age1,60.74824
b0iR,0.00000
b1iR,-0.00000
LatVentR1,13777.27703
Vfixed1R,13339.96594
Vfixed1diffR,437.31108
VdevR,471.60451
errorR,-34.29343


### Check summary statistics

In [21]:
pd_df = dff.copy()
pd_df.describe(include=[np.number]).round(4).T

,count,mean,std,min,25%,50%,75%,max
id,74.0,37.5000,21.5058,1.0000,19.2500,37.5000,55.7500,74.0000
Age1,74.0,60.7482,7.2548,46.6600,55.6125,59.8750,66.3925,77.6300
Age2,74.0,64.2839,7.2110,50.1100,59.3175,63.5550,69.2300,81.3300
Age3,74.0,66.8126,7.1855,52.4600,61.8600,66.0200,72.0925,84.0500
b0iL,74.0,-0.0000,19104.1076,-92438.6924,-6894.5245,3257.9935,10096.4873,35796.8808
b1iL,74.0,0.0000,370.0545,-736.2918,-219.9263,-84.9836,213.9539,1605.3663
beta0plusb0iL,74.0,-12057.6213,19104.1076,-104496.3137,-18952.1458,-8799.6278,-1961.1340,23739.2595
beta1plusb1iL,74.0,428.7273,370.0545,-307.5645,208.8009,343.7437,642.6811,2034.0936
b0iR,74.0,0.0000,8988.7901,-32640.0136,-3525.8863,2838.3733,6388.6144,12136.8345
b1iR,74.0,-0.0000,248.1606,-340.3251,-170.6087,-72.1386,96.2811,846.8899


In [22]:
pd.DataFrame([pd_df['id'], pd_df['Age3'], pd_df['VdevL'], pd_df['VdevR']]).round(4)

,0,1,2,3,4,5,6,7,8,9,...,64,65,66,67,68,69,70,71,72,73
id,1.0000,2.0000,3.0000,4.0000,5.0000,6.0000,7.0000,8.0000,9.0000,10.0000,...,65.0000,66.0000,67.0000,68.0000,69.0000,70.0000,71.0000,72.0000,73.0000,74.0000
Age3,62.6200,55.0500,80.5900,62.6800,69.5300,66.8200,64.8700,75.4700,65.5900,60.6300,...,67.8800,63.8000,77.1700,66.0100,67.8200,62.4000,73.0000,71.5900,70.7200,70.8300
VdevL,-7043.9213,2022.8338,9742.1408,9671.2606,1969.9293,1359.3110,6505.0743,-5823.4591,4081.0741,928.0691,...,6117.3847,-7586.6683,-8080.9075,-2518.3479,300.2172,-1753.0873,-8167.0147,-7840.1456,-2120.1255,-207.2701
VdevR,-5184.3248,1484.9150,15864.7936,3369.2981,-1779.9775,418.5232,-782.9705,-5848.0511,7780.1374,-918.0942,...,9400.7515,-6052.0181,-7844.8193,-3304.4677,-3830.3640,-1490.9218,-9949.6286,-7391.9333,-4875.8240,-1471.2448


### Select variables for analysis

In [23]:
pd_df.columns

Index(['id', 'Gender', 'Age1', 'Age2', 'Age3', 'b0iL', 'b1iL', 'beta0plusb0iL',
       'beta1plusb1iL', 'b0iR', 'b1iR', 'beta0plusb0iR', 'beta1plusb1iR',
       'LatVentL1', 'LatVentL2', 'LatVentL3', 'LatVentR1', 'LatVentR2',
       'LatVentR3', 'RI3', 'VdevL', 'Vfixed1L', 'Vfixed1diffL', 'VdevR',
       'Vfixed1R', 'Vfixed1diffR', 'errorL', 'errorR'],
      dtype='object')

In [24]:
myvars = [
    'id',
    'Gender',
    'Age3',
    'b1iL',
    'b1iR',
    'VdevL',
    'VdevR',
    'RI3'
]

In [25]:
pd_df.groupby("Gender")[myvars].describe(include="all", percentiles = [0.5]).round(4).T

Gender            F        M
id  count      48.0     26.0
    unique      NaN      NaN
    top         NaN      NaN
    freq        NaN      NaN
    mean    39.8125  33.2308
...             ...      ...
RI3 mean    56.0208  58.7692
    std     14.9503  13.6185
    min        35.0     40.0
    50%        52.5     57.0
    max       102.0     94.0

[72 rows x 2 columns]

### $\LaTeX$ related

In [26]:
#print(pd_df.groupby("Gender")[myvars].describe(include="all", percentiles = [0.5]).round(2).T.to_latex())

#### Cut and paste LaTeX code to new cell
%%latex  cell

substitute tabular with array as "MathJax doesn't implement tabular", and toprule, midrule with hline <br>
https://github.com/mathjax/mathjax-docs/wiki/LaTeX-Tabular-environment


### Save lmer features

In [27]:
dfmri = pd_df.copy()
dfmri = dfmri[myvars].drop('id', axis=1)
dfmri.head().round(4)

,Gender,Age3,b1iL,b1iR,VdevL,VdevR,RI3
0,F,62.62,-403.1060,-255.8375,-7043.9213,-5184.3248,66
1,M,55.05,-192.2795,18.4452,2022.8338,1484.9150,85
2,M,80.59,504.1641,431.9485,9742.1408,15864.7936,71
3,M,62.68,242.6547,143.7001,9671.2606,3369.2981,41
4,M,69.53,-535.0672,-119.3152,1969.9293,-1779.9775,62


In [28]:
# Save the lmer feature data to the present repository as .csv
if eTIV_NORMALIZED:
    dfmri.to_csv('../results/02_lvv_ri_new_R_lmer_feature_data_eTIV_norm.csv', header=True, index=False)
else:
    dfmri.to_csv('../results/02_lvv_ri_new_R_lmer_feature_data.csv', header=True, index=False)   

# END